# Librerias

In [121]:
### Import necessary libraries
import time
import random
import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By
from selenium.webdriver import Firefox
from urllib.parse import urlparse, parse_qsl, urlencode, urlunparse
import json
import pickle


# Funciones para el proyecto

In [245]:
def get_random_user_agent():
    HEADERS_LIST = [
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/",
            "Connection": "keep-alive",
            "Cookie": "CONSENT=YES+1; NID=511=exampleSessionId123abc"
        },
        {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.8",
            "Referer": "https://duckduckgo.com/",
            "Connection": "keep-alive",
            "Cookie": "D=do_not_track; example_cookie_id=macSafariSession987"
        },
        {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9",
            "Accept-Language": "en-GB,en;q=0.8",
            "Referer": "https://www.bing.com/",
            "Connection": "keep-alive",
            "Cookie": "session=linux_session_xyz789; logged_in=true"
        },
        {
            "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://m.facebook.com/",
            "Connection": "keep-alive",
            "Cookie": "fb_locale=en_US; datr=mobileSafariFakeSession123"
        }
    ]
    return random.choice(HEADERS_LIST)

In [246]:
def setup_driver(headless=False):
    headers = get_random_user_agent()

    profile = FirefoxProfile()
    profile.set_preference("general.useragent.override", headers["User-Agent"])
    profile.set_preference("intl.accept_languages", headers["Accept-Language"])

    options = Options()
    options.profile = profile
    if headless:
        options.add_argument("--headless")

    driver = Firefox(options=options)
    driver.set_page_load_timeout(10)
    
    # Navegar desde un referer simulado
    driver.execute_script(f"window.location.href = '{headers['Referer']}';")

    return driver

In [247]:
### Sistema anti-bot (comportamiento humano)
def simulate_human_behavior(min_delay=1, max_delay=3, verbose=False):
    """Simula comportamiento humano con sleeps aleatorios."""
    delay = round(random.uniform(min_delay, max_delay), 2)
    if verbose:
        print(f"Esperando {delay}s...")
    time.sleep(delay)


In [248]:
### Obtener lista de practice areas para aplicar filtros
def extract_trimmed_set(series):
    """
    Recibe una Serie de pandas con valores separados por comas,
    elimina NaN, divide por ',' y devuelve un set de elementos únicos y limpios.
    """
    values = series.dropna().astype(str)
    trimmed_set = set()
    for val in values:
        items = [item.strip() for item in val.split(',')]
        trimmed_set.update(items)
    return trimmed_set


In [249]:
### Obtener paginas totales por cada busqueda una vez aplicado el filtro 
def generate_paginated_urls(url,driver,verbose=False):
    # Encontrar numero de paginas 
    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,'ul.right')))
        simulate_human_behavior(1,2,verbose=False)
        pages_tab = driver.find_element(By.CSS_SELECTOR,'ul.right')       
        if pages_tab:
            pages = pages_tab.find_elements(By.TAG_NAME,"li")
            pages = pages[1].text
            pages = int(pages.split()[-1])
    except Exception as e: 
        #print(f"Solo hay una pagina")
        pages = 1

    #print(f"total pages:{pages}")
    if verbose:
        print(f"paginas en esta url: {pages}")
    parsed = urlparse(url)
    query_list = parse_qsl(parsed.query)  # esto mantiene el orden de los parámetros

    urls = []
    for i in range(1, pages + 1):
        # Creamos una nueva lista sin el parámetro page
        new_query = [(k, v) for (k, v) in query_list if k != "page"]
        # Si no es la primera página, agregamos page=N
        if i != 1:
            new_query.append(("page", str(i)))
        # Reconstruimos la query string manteniendo el orden
        new_query_string = urlencode(new_query)
        # Reconstruimos la URL
        new_url = urlunparse((parsed.scheme, parsed.netloc, parsed.path, '', new_query_string, ''))
        urls.append(new_url)

    return urls


In [250]:
### crawler
def crawler(url, driver,verbose=False,get_driver=True):   
    try:
        list_urls = []
        urls_unicos = set()  # Usar un set para almacenar URLs únicas
        if get_driver:
            driver.get(url)
        time.sleep(1)
        # buscar contenedor con info
        WebDriverWait(driver, 25).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.results__title')))
        result_title = driver.find_element(By.CSS_SELECTOR, '.results__title')
        if result_title:
            WebDriverWait(driver, 25).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.results__result-row'))) 
            attorney_table = driver.find_element(By.CSS_SELECTOR, '.results__result-row')
            abogados = attorney_table.find_elements(By.XPATH, '(//div[@class="row collapse card__content"])')
            if verbose:
                print("se encontro la tabla de abogados")
                print(len(abogados))
                print(abogados[0])

            for ul in abogados:
                try:
                    WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.TAG_NAME, 'ul')))
                    enlaces = ul.find_elements(By.TAG_NAME, 'a')
                    href_attorney = enlaces[1].get_attribute('href')         # Obtiene el atributo href
                    if verbose:
                            print(href_attorney)
                    if href_attorney not in urls_unicos:
                        urls_unicos.add(href_attorney)
                        list_urls.append(href_attorney)
                except:
                    print("No se encontró <a> en este <li>")
                    pass
        if verbose:
            print(driver.current_url)
            print(len(list_urls))
    except:
        print("problemas con el crawler")
    return list_urls 


In [311]:
### función para obtener el HTML
def get_html(driver,url,verbose=False):
    driver.get(url)
    time.sleep(random.uniform(3, 5))
    if verbose:
        print(f"accediendo a: {url}")
    WebDriverWait(driver, 15).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'body > div.off-canvas-wrap > div > div.attorney-profile-content.profile-content')))
    simulate_human_behavior(1, 3, verbose=False)
    html = driver.page_source
    return html


In [319]:
### funcion Scraper pagina por pagina abogados
def scrape_abogados(lista_urls):
    resultados = []
    count = 2442
    for url in lista_urls:
        driver = setup_driver(headless=True)
        info_attorney = {}
        try:
            print(f"accediendo a {count}: {url}")
            html = get_html(driver,url)
            soup = BeautifulSoup(html, 'html.parser')

            profile_content = soup.select_one('div.attorney-profile-content.profile-content')
            if not profile_content:
                print(f"Perfil no encontrado para {url}")
                pass

            # Obtencion de elementos princiaples de cada pagina
            attorney_card = profile_content.select_one(".masthead-content")
            firm = attorney_card.find("span")
            if firm:
                firm = firm.get_text(strip=True)
            url_martindale_firm = attorney_card.find("a")["href"]

            # En caso de que no haya un link de firma, el primer link sera la caja de rewiew que redirige a la misma pagina, por eso el #
            if "#" in url_martindale_firm:
                url_martindale_firm = None

            # Estructura del diccionario
            info_attorney["url_martindale"] = url #URL interna de martindale para cada abogado
            info_attorney["name"] = profile_content.find('h1').get_text(strip=True) #Nombre del abogado
            info_attorney["firm"] = firm
            info_attorney["url_firm_martindale"] = url_martindale_firm

            # Busqueda de la sección de links de contacto, no todas las paginas tienen esta sección. por eso se usa try/except 
            try: 
                web_bar = profile_content.select_one("section.sticky-menu-all:nth-child(2) > nav:nth-child(1)")
                website_firm = web_bar.select('a')[1]["href"]
                info_attorney["website_firm"] = website_firm if website_firm else None
            except Exception as e:
                #print(f"Error al extraer website: {e}")
                info_attorney["website"] = ""

            #Extraccion de los tags relevantes de cada pagina
            main_content_table = soup.find_all("div", class_="small-12 columns")[0]
            for section_div in main_content_table.find_all("div", recursive=False):
                content_div = section_div.find("div", class_="toggle-area__content")
                h2_key = section_div.find("h2") #Titulo de sección por separado
                if h2_key and content_div:
                    h2_key = h2_key.get_text(strip=True)
                    if "Areas of Practice" in h2_key:
                        h2_key = "Areas of Practice"
                    #print(h2_key)
                    info_attorney[h2_key] = "" #inicializar keys que se encuentran

                    # Biography and practice areas 
                    child_text = content_div.find('div', class_='truncate-text')
                    if child_text:
                        info_attorney[h2_key] = child_text.get_text(strip=True)
                    Aoe_area = content_div.find('div', class_='row collapse')
                    if Aoe_area:
                        ul_info = Aoe_area.find('ul')
                        if ul_info:
                            items = [li.get_text(strip=True) for li in ul_info.find_all('li')]
                            info_attorney[h2_key] = ', '.join(items)

                    # Education and credentials
                    EnC_area = content_div.find_all(
                        lambda tag: tag.name == 'div' and 
                        all(cls in tag.get('class', []) for cls in ['row', 'collapse', 'experience-section', 'clearfix'])
                    )
                    for text in EnC_area:
                        text_info = text.get_text(strip=True)
                        for line in text_info.split('\n'):
                            if ':' in line:
                                k, v = map(str.strip, line.split(":", 1))
                                info_attorney[k] = v
                            else:
                                info_attorney[h2_key] = line.strip()

                    # Peer rewiew - Pendiente
                    PrA_section = content_div.find("section", class_="profile-peer-reviews")
                    if PrA_section: 
                        rewiew_area = PrA_section.find('div', class_="review-top review-top-peer-reviews")
                        rewiew_list = PrA_section.find('div', class_="attorney-reviews-list attorney-peer-reviews-list")
                        if rewiew_area and rewiew_list:
                            text_rewiew_area = rewiew_area.find_all('span')[2].get_text().strip().splitlines()
                            if len(text_rewiew_area) == 2:
                                stars = text_rewiew_area[0].split("/")[0].strip()
                                reviews = text_rewiew_area[1].strip("()").split()[0]
                                info_attorney["profile_peer_review_star"] = stars
                                info_attorney["profile_peer_review_count"] = reviews
                            else:
                                stars = text_rewiew_area[0].split("/")[0].strip()
                                info_attorney["profile_peer_review_star"] = stars
                    # Location
                    firm_offices = content_div.find('ul', id='firmOffices')
                    if firm_offices:
                        location_items = firm_offices.find_all('div', class_='office-address')
                        if location_items:
                            info_attorney["Location"] = location_items[0].get_text(strip=True)
            if info_attorney:
                resultados.append(info_attorney)
            count += 1 
            driver.quit()
        except TimeoutException:
            print("Loading took too much time!")
            print(f"error en iter {count}")
            driver.quit()
            pass
            return resultados
        except Exception as e:
            print(f"Failed to load page due to error: {e}")
            print(f"error en iter {count}")
            driver.quit()
            return resultados
        except NoSuchElementException:
            driver.quit()
            print(f"error en iter {count}")
            return resultados
        driver.quit()
    # Resultado ya es una lista de diccionarios
    return resultados


# Obtención lista "practice areas"

In [63]:
### Generador de strings
def generar_permutaciones(min_len=2, max_len=2):
    """
    Genera permutaciones de letras minúsculas desde longitud min_len hasta max_len.
    
    Por defecto, genera solo combinaciones de 2 caracteres como: aa, ab, ..., zz.
    """
    letras = string.ascii_lowercase  # 'abcdefghijklmnopqrstuvwxyz'
    resultados = []
    
    for longitud in range(min_len, max_len + 1):
        for combinacion in itertools.product(letras, repeat=longitud):
            resultados.append(''.join(combinacion))
    
    return resultados

In [64]:
### llamado de funcion para generar strings 
perms = generar_permutaciones(2, 2)
print(perms[-10:])  # ['aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj']
print(f"Total permutaciones de longitud 2: {len(perms)}")  # 26^2 = 676

['zq', 'zr', 'zs', 'zt', 'zu', 'zv', 'zw', 'zx', 'zy', 'zz']
Total permutaciones de longitud 2: 676


In [81]:
#all_info = pd.DataFrame() # Data frame para almacenar información
url_inicial = 'https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto%20Rico%2C%20USA'
lista_urls = []
urls_per_filter = [] 
datos_preliminares = []


In [82]:
### Limpieza set Areas of practice - insumo para nuevo crowler
# Esta parte puede ser una función que se aplique antes de entrar a hacer las busquedas 

practices_areas_compilate = set() # Se crea un set con las posibles opciones
len(perms) # Toma el set de todas las posibles practice areas, escritas por abogado
url_base = url_inicial ## CAMBIAR MAS ADELANTE
driver = setup_driver()
wait = WebDriverWait(driver, 15)
driver.get(url_base)

for area in perms: 
    wait.until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="filtersContainer"]'))) 
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="practiceAreas"]')))
    #print("es clickeable")
    searchBar = driver.find_element(By.ID, 'practiceAreas')
    searchBar.clear()
    time.sleep(2)
    searchBar.send_keys(area)
    wait.until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "ul.ui-autocomplete li.ui-menu-item div.ui-menu-item-wrapper")))
    suggestions = driver.find_elements(By.CSS_SELECTOR, "ul.ui-autocomplete li.ui-menu-item div.ui-menu-item-wrapper")
    if suggestions:
        for sugestion in suggestions:     
            if "No Areas Of Practice matching" in sugestion.text:
                pass
            if sugestion.text not in practices_areas_compilate:
                practices_areas_compilate.add(sugestion.text)
                print (sugestion.text)
    searchBar.clear()

len(practices_areas_compilate)


No Areas Of Practice matching
Administrative Law
Admiralty and Maritime Law
Advertising and Marketing
Administrative Hearings and Appeals
Adoptions
Affirmative Action
Affordable Housing
Agency and Distributorships
Agricultural Law
Airplane Crash Litigation
Alternative Dispute Resolution
Americans with Disabilities Act
Antitrust and Trade Regulation
Animal Law
Appellate Practice
Arbitration
Architects and Engineers Liability
Asset Protection
Asylum
Asbestos Litigation
Assault
Automotive Products Liability
Automobile Insurance
Aviation Law
Banking Law
Bankruptcy
Bankruptcy Chapter 7
Bankruptcy Chapter 11
Bankruptcy Chapter 13
Biotechnology
Birth Injuries
Boating Accidents
Breach of Contract
Business Law
Business Litigation
Business Formation
Business Planning
Buying and Selling of Businesses

Casinos and Gambling
Cerebral Palsy
Child Custody
Child Support
Civil Litigation
Civil Practice
Civil Rights
Citizenship
Class Actions
Closely Held Business Law
Commercial Law
Corporate Law
Commerci

216

In [ ]:
practices_areas_compilate = list(practices_areas_compilate)
practices_areas_compilate.pop(0)
practices_areas_compilate # Se elimino el primer elemento, tener en cuenta la proxima. 


['Contracts',
 'Civil Litigation',
 'Head and Spinal Injuries',
 'Partnership Law',
 'Education Law',
 'DUI and DWI',
 'Shipping',
 'Birth Injuries',
 'Business Law',
 'Drug and Medical Device Litigation',
 'General Practice',
 'Foreclosures',
 'Buying and Selling of Businesses',
 'Resorts and Leisure',
 'Sports Law',
 'Family Mediation',
 'RICO',
 'Divorce',
 'Joint Ventures',
 'Utility Law',
 'Living Wills',
 'Family and Medical Leave Act',
 'Religious Institutions',
 'Unfair Competition',
 'Law Enforcement',
 'Discrimination',
 'Property Law',
 'Government Relations',
 'Inheritance',
 'Bankruptcy Chapter 13',
 'Litigation',
 'Insurance',
 'Libel, Slander and Defamation',
 'Fidelity and Surety',
 'Labor and Employment',
 'Employment Contracts',
 'Americans with Disabilities Act',
 'Wetlands',
 'Wills and Probate',
 'Workers Compensation',
 'Limited Liability Company Law',
 'Breach of Contract',
 'Environmental Law',
 'Housing Law',
 'Bankruptcy',
 'Affirmative Action',
 'Railroad Acc

In [ ]:
### Almacenar set de pracite areas 
with open("practice_area_set.pkl", "wb") as f:
    pickle.dump(practices_areas_compilate, f)

In [112]:
with open("urls_region_PR.pkl", "rb") as f:
    urls_total = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
    # Esta variable va a ser mas general por lo que se va a emplear mas adelante. o asignar a la primera corrida. 
urls_total

[]

In [113]:
### Extracción multiples urls abogados aplicando filtros con set de "practice area" que si funcionan generan sugerencias en el buscador

# clean va a ser una lista de practice area que ya paso por un proceso de filtrado. 
#url_base = 'https://www.martindale.com/search/attorneys/?term=Puerto%20Rico%2C%20USA' #Va a cambiar mas adelante
url_base = 'https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto%20Rico%2C%20USA&loc=1'
try:
    for area in tqdm.tqdm(practices_areas_compilate):
        ### configuración del webdriver
        driver = setup_driver(headless=True)
        wait = WebDriverWait(driver, 25)
        driver.get(url_base)
        simulate_human_behavior(0.5,3,verbose=False)

        ### Elementos clickeables y request
        # Ubicar barra de busqueda "Filters - Practice Areas"
        wait.until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="filtersContainer"]'))) 
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="practiceAreas"]')))
        searchBar = driver.find_element(By.ID, 'practiceAreas')
        searchBar.clear()
        
        # Envio de texto "practice area" para aplicar filtro y tener nuevos resultados
        searchBar.send_keys(area)
        simulate_human_behavior(1,2,verbose=False)
        suggestion = wait.until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "ul.ui-autocomplete li.ui-menu-item div.ui-menu-item-wrapper")))

        # Busqueda de practice area - Si hay una sugerencia, se da click. 
        suggestion.click() # ----> Entra a nueva pagina con filtro.
        simulate_human_behavior(1,2,verbose=False) 
        current_url = driver.current_url # Actualizar la url para obtener N° de paginas y urls de paginas
        
        # links y N° de paginas una vez aplicado el filtro practice area
        urls = generate_paginated_urls(current_url, driver,verbose=True)
        for url in urls:
            print(f"accediendo a: {url}")
            urls_per_page = crawler(url,driver,verbose=False)
            if len(urls_per_page) == 0:
                urls_per_page = crawler(url,driver,verbose=False)
                time.sleep(2)
            if len(urls_per_page) > 1:
                for link in urls_per_page:
                    if not link in urls_total:
                        urls_total.append(link)
                        print(link)
                    else: 
                        pass
                # La lista urls_per_filter ya tiene 500 valores debido a la primera iteración del proceso. 
                # despues de cada filtro, hay links repetidos, se intenta que no hayan duplicados y sea la lista total por estado. 
        driver.quit()
except TimeoutException:
    print("Loading took too much time!")
    driver.quit()
    pass
except Exception as e:
    print(f"Failed to load page due to error: {e}")
    driver.quit()
driver.quit()
print(len(urls_per_filter))


  0%|          | 0/213 [00:00<?, ?it/s]

paginas en esta url: 13
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiQ29udHJhY3RzIl19
https://www.martindale.com/attorney/ramon-mellado-gonzalez-1572913/
https://www.martindale.com/attorney/maximiliano-a-pla-mendez-25744746/
https://www.martindale.com/attorney/juan-a-cuyar-cobb-1571691/
https://www.martindale.com/attorney/edgardo-j-hernandez-oharriz-esq-158870123/
https://www.martindale.com/attorney/roberto-c-rodriguez-poventud-1571906/
https://www.martindale.com/attorney/joseph-d-lento-168969230/
https://www.martindale.com/attorney/jairo-jay

  0%|          | 1/213 [00:28<1:42:02, 28.88s/it]

paginas en esta url: 17
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiQ2l2aWwgTGl0aWdhdGlvbiJdfQ%3D%3D
https://www.martindale.com/attorney/raquel-santos-ramos-45015561/
https://www.martindale.com/attorney/arytza-y-martinez-3115340/
https://www.martindale.com/attorney/maria-victoria-munera-pascual-1573325/
https://www.martindale.com/attorney/maritere-colon-dominguez-13986963/
https://www.martindale.com/attorney/oreste-ricardo-ramos-2170248/
https://www.martindale.com/attorney/juan-eliza-burgos-13781059/
https://www.martindale.com/attorney/osval

  1%|          | 2/213 [01:03<1:54:11, 32.47s/it]

...

paginas en esta url: 1
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiQWRtaW5pc3RyYXRpdmUgSGVhcmluZ3MgYW5kIEFwcGVhbHMiXX0%3D


100%|█████████▉| 212/213 [1:27:23<00:23, 23.49s/it]

paginas en esta url: 6
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiVHJ1c3RzIGFuZCBFc3RhdGVzIl19
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcn

100%|██████████| 213/213 [1:27:41<00:00, 24.70s/it]

0


In [116]:
urls_total

['https://www.martindale.com/attorney/ramon-mellado-gonzalez-1572913/',
 'https://www.martindale.com/attorney/maximiliano-a-pla-mendez-25744746/',
 'https://www.martindale.com/attorney/juan-a-cuyar-cobb-1571691/',
 'https://www.martindale.com/attorney/edgardo-j-hernandez-oharriz-esq-158870123/',
 'https://www.martindale.com/attorney/roberto-c-rodriguez-poventud-1571906/',
 'https://www.martindale.com/attorney/joseph-d-lento-168969230/',
 'https://www.martindale.com/attorney/jairo-jay-mellado-villarreal-1572914/',
 'https://www.martindale.com/attorney/ms-helaine-a-gregory-321442/',
 'https://www.martindale.com/attorney/pedro-f-silva-ruiz-1574066/',
 'https://www.martindale.com/attorney/gustavo-a-chico-barris-3318975/',
 'https://www.martindale.com/attorney/ms-lourdes-m-rodriguez-morera-46209566/',
 'https://www.martindale.com/attorney/nestor-duran-gonzalez-1573227/',
 'https://www.martindale.com/attorney/johanny-martinez-batista-2092244/',
 'https://www.martindale.com/attorney/ramon-ant

In [115]:
with open("urls_region_PR.pkl", "wb") as f:
    pickle.dump(urls_total, f)

In [ ]:
### Prueba scrapper 1 url
#url = "https://www.martindale.com/attorney/carlos-villegas-del-valle-301593670/#officeLocation"
#url = "https://www.martindale.com/attorney/rafael-g-rivera-rosario-1572639/"
url = "https://www.martindale.com/attorney/jairo-jay-mellado-villarreal-1572914/"

resultados = []
count = 1
driver = setup_driver()
wait = WebDriverWait(driver, 15)
info_attorney = {}
inicio = time.time()
for i in range(1):
    try:
        html = get_html(driver,url)
        soup = BeautifulSoup(html, 'html.parser')
        
        profile_content = soup.select_one('div.attorney-profile-content.profile-content')
        if not profile_content:
            print(f"Perfil no encontrado para {url}")
            pass

        # Obtencion de elementos princiaples de cada pagina
        attorney_card = profile_content.select_one(".masthead-content")
        firm = attorney_card.find("span").get_text()
        url_martindale_firm = attorney_card.find("a")["href"]

        # En caso de que no haya un link de firma, el primer link sera la caja de rewiew que redirige a la misma pagina, por eso el #
        if "#" in url_martindale_firm:
            url_martindale_firm = None

        # Estructura del diccionario
        info_attorney["url_martindale"] = url #URL interna de martindale para cada abogado
        info_attorney["name"] = profile_content.find('h1').get_text(strip=True) #Nombre del abogado
        info_attorney["firm"] = firm
        info_attorney["url_firm_martindale"] = url_martindale_firm

        # Busqueda de la sección de links de contacto, no todas las paginas tienen esta sección. por eso se usa try/except 
        try: 
            web_bar = profile_content.select_one("section.sticky-menu-all:nth-child(2) > nav:nth-child(1)")
            website_firm = web_bar.select('a')[1]["href"]
            info_attorney["website_firm"] = website_firm if website_firm else None
        except Exception as e:
            #print(f"Error al extraer website: {e}")
            info_attorney["website"] = ""

        #Extraccion de los tags relevantes de cada pagina
        main_content_table = soup.find_all("div", class_="small-12 columns")[0]
        for section_div in main_content_table.find_all("div", recursive=False):
            content_div = section_div.find("div", class_="toggle-area__content")
            h2_key = section_div.find("h2") #Titulo de sección por separado
            if h2_key and content_div:
                h2_key = h2_key.get_text(strip=True)
                if "Areas of Practice" in h2_key:
                    h2_key = "Areas of Practice"
                print(h2_key)
                info_attorney[h2_key] = "" #inicializar keys que se encuentran

                # Biography and practice areas 
                child_text = content_div.find('div', class_='truncate-text')
                if child_text:
                    info_attorney[h2_key] = child_text.get_text(strip=True)
                Aoe_area = content_div.find('div', class_='row collapse')
                if Aoe_area:
                    ul_info = Aoe_area.find('ul')
                    if ul_info:
                        items = [li.get_text(strip=True) for li in ul_info.find_all('li')]
                        info_attorney[h2_key] = ', '.join(items)

                # Education and credentials
                EnC_area = content_div.find_all(
                    lambda tag: tag.name == 'div' and 
                    all(cls in tag.get('class', []) for cls in ['row', 'collapse', 'experience-section', 'clearfix'])
                )
                for text in EnC_area:
                    text_info = text.get_text(strip=True)
                    for line in text_info.split('\n'):
                        if ':' in line:
                            k, v = map(str.strip, line.split(":", 1))
                            info_attorney[k] = v
                        else:
                            info_attorney[h2_key] = line.strip()
                
                # Peer rewiew - Pendiente
                PrA_section = content_div.find("section", class_="profile-peer-reviews")
                if PrA_section: 
                    rewiew_area = PrA_section.find('div', class_="review-top review-top-peer-reviews")
                    rewiew_list = PrA_section.find('div', class_="attorney-reviews-list attorney-peer-reviews-list")
                    if rewiew_area and rewiew_list:
                        text_rewiew_area = rewiew_area.find_all('span')[2].get_text().strip().splitlines()
                        stars = text_rewiew_area[0].split("/")[0].strip()   
                        reviews = text_rewiew_area[1].strip("()").split()[0]
                        info_attorney["profile_peer_review_star"] = stars
                        info_attorney["profile_peer_review_count"] = reviews
                    
                # Location
                firm_offices = content_div.find('ul', id='firmOffices')
                if firm_offices:
                    location_items = firm_offices.find_all('div', class_='office-address')
                    if location_items:
                        info_attorney["Location"] = location_items[0].get_text(strip=True)
        if info_attorney:
            resultados.append(info_attorney)
        count += 1 
        driver.quit()
        False
    except (NoSuchElementException, TimeoutException) as e:
        print(f"Error procesando {url}: {str(e)}")
        driver.quit()
    driver.quit()
    final = time.time() - inicio
    print(f"Tiempo por iteración: {final}")


# Corrida final, extracción de datos lista de urls final con filtros aplicados

In [117]:
### carga de listado de urls
with open("urls_region_PR.pkl", "rb") as f:
    urls_total = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
    # Esta variable va a ser mas general por lo que se va a emplear mas adelante. o asignar a la primera corrida. 
print(len(urls_total))
datos_PR = []
datos_parciales = []


2464


In [320]:
datos_parciales = scrape_abogados(urls_total[2442:])
#Si falla, revisar en que iteración fue para continuar en la lista de urls_totales


accediendo a 2442: https://www.martindale.com/attorney/hector-m-laffitte-11861386/
accediendo a 2443: https://www.martindale.com/attorney/ms-lilli-w-marder-esq-799259/
accediendo a 2444: https://www.martindale.com/attorney/charles-wayne-jimenez-1574274/
accediendo a 2445: https://www.martindale.com/attorney/marisa-brugueras-1573031/
accediendo a 2446: https://www.martindale.com/attorney/myria-a-rodriguez-28972826/
accediendo a 2447: https://www.martindale.com/attorney/roberto-r-ramos-antonmattei-2849199/
accediendo a 2448: https://www.martindale.com/attorney/melissa-lopez-diaz-2849165/
accediendo a 2449: https://www.martindale.com/attorney/melvin-a-padilla-feliciano-1573675/
accediendo a 2450: https://www.martindale.com/attorney/janitza-m-garcia-marrero-3666579/
accediendo a 2451: https://www.martindale.com/attorney/laura-gonzalez-bothwell-1572220/
accediendo a 2452: https://www.martindale.com/attorney/florencio-g-vega-diaz-jr-1574121/
accediendo a 2453: https://www.martindale.com/atto

In [321]:
datos_PR = datos_PR + datos_parciales

In [322]:
len(datos_PR)

2677

In [323]:
with open("data_attornies.pkl", "wb") as f:
    pickle.dump(datos_PR, f)
len(datos_PR)

2677

In [326]:
with open("data_attornies.pkl", "rb") as f:
    A = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
with open("practice_area_set.pkl", "rb") as f:
    B = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
with open("urls_region_PR.pkl", "rb") as f:
    C = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.

len(B)

216